#Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.


In [1]:
!apt install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 32 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lib

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient('mongodb://127.0.0.1:27017/')

In [5]:
my_db = client['base_vacancies_analyst']
collection_vacancies = my_db['vacancies']

##Заполним базу данных из файла ДЗ №3

In [6]:
import json
with open('dict_vacancy.json', "r") as json_file:
    data_file = json.load(json_file)

In [25]:
for vacancy in data_file:
  document = data_file[vacancy]
  my_db.vacancies.insert_one(document)

In [8]:
client.list_database_names()

['admin', 'base_vacancies_analyst', 'local']

In [24]:
for coll in my_db.list_collection_names():
    print(coll)

vacancies


In [26]:
num_vacancies = my_db.vacancies.count_documents({})
print(num_vacancies)

100


In [12]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pprint import pprint

##Реализуем функцию добавления новых вакансий в базу данных путём парсинга первой страницы сайта hh.ru

In [13]:
def add_vacancies(url, headers):
  response = requests.get(url = url, headers = headers)
  print(response.status_code)
  dom = bs(response.text, 'html.parser')
  tag_vacancy = dom.find_all('div', class_= 'vacancy-serp-item-body__main-info')
  print(f'считано - {len(tag_vacancy)} вакансий')
  site_name = 'hh.ru'
  number_new_vacancies = 0
  for vacancy in tag_vacancy:
    vacancy_link =  vacancy.find('a', attrs={'class': 'serp-item__title', 'data-qa': 'serp-item__title'}).get('href')
    if my_db.vacancies.count_documents({'ссылка на вакансию': vacancy_link}) == 0:
      vacancy_title = vacancy.find('a', attrs={'class': 'serp-item__title', 'data-qa': 'serp-item__title'}).string
      if vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}) == None:
        min_compensation, max_compensation, currency = None, None, None
      else:
        compensation = vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}).text.replace(' ', '').split()
        if compensation[0] == 'от':
          min_compensation = int(compensation[1])
          max_compensation = None
          currency = compensation[2]
        elif compensation[0] == 'до':
          min_compensation = None
          max_compensation = int(compensation[1])
          currency = compensation[2]
        else:
          min_compensation = int(compensation[0])
          max_compensation = int(compensation[2])
          currency = compensation[3]
      my_db.vacancies.insert_one({
          'должность': vacancy_title,
          'минимальная зарплата' : min_compensation,
          'максимальная зарплата' : max_compensation,
          'валюта' : currency,
          'ссылка на вакансию' : vacancy_link,
          'сайт' : site_name
           })
      number_new_vacancies +=1
  print(f'добавлено {number_new_vacancies} новых вакансий')  
      

In [27]:
add_vacancies('https://hh.ru/vacancies/analitik-dannyh?page=num_page', {
    'User-Agent':	'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
})

200
считано - 50 вакансий
добавлено 30 новых вакансий


In [28]:
num_vacancies = my_db.vacancies.count_documents({})
print(num_vacancies)

130


#* Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [29]:
for doc in my_db.vacancies.find({'$and': [{'минимальная зарплата':{'$gt': 200000}}, {'максимальная зарплата':{'$gt': 200000}}]}):
  pprint(doc)

{'_id': ObjectId('63209047eb05959038e17249'),
 'валюта': 'руб.',
 'должность': 'Системный аналитик | Systems Analyst',
 'максимальная зарплата': 370000,
 'минимальная зарплата': 250000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69481665?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e17258'),
 'валюта': 'руб.',
 'должность': 'Ведущий аналитик',
 'максимальная зарплата': 500000,
 'минимальная зарплата': 250000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/67702365?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e1725c'),
 'валюта': 'руб.',
 'должность': 'Системный аналитик',
 'максимальная зарплата': 250000,
 'минимальная зарплата': 235000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69569042?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0

In [30]:
my_db.vacancies.count_documents({'$and': [{'минимальная зарплата':{'$gt': 150000}}, {'максимальная зарплата':{'$gt': 200000}}]})

6

#* Любая аналитика. Например matching ваканский с разных площадок
### Выведем все вакансии без указания сумм вознаграждений.

In [31]:
for doc in my_db.vacancies.find({'$nor': [{'минимальная зарплата':None}, {'максимальная зарплата':None}]}):
  pprint(doc)

{'_id': ObjectId('63209047eb05959038e17241'),
 'валюта': 'руб.',
 'должность': 'Аналитик данных / Data Analyst',
 'максимальная зарплата': 300000,
 'минимальная зарплата': 150000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69360960?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e17242'),
 'валюта': 'руб.',
 'должность': 'Аналитик данных / Data Analyst',
 'максимальная зарплата': 90000,
 'минимальная зарплата': 60000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69647954?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e17243'),
 'валюта': 'руб.',
 'должность': 'Аналитик BI',
 'максимальная зарплата': 200000,
 'минимальная зарплата': 160000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69592342?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%

###Выведем вакансии, в названии которых присутствует слово "Ведущий".

In [32]:
for doc in my_db.vacancies.find({ 'должность': { "$regex": 'Ведущий'} }):
  pprint(doc)

{'_id': ObjectId('63209047eb05959038e17258'),
 'валюта': 'руб.',
 'должность': 'Ведущий аналитик',
 'максимальная зарплата': 500000,
 'минимальная зарплата': 250000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/67702365?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e17259'),
 'валюта': 'руб.',
 'должность': 'Ведущий аналитик данных (Департамент закупок)',
 'максимальная зарплата': 300000,
 'минимальная зарплата': None,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/49495729?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85'}
{'_id': ObjectId('63209047eb05959038e1728a'),
 'валюта': 'руб.',
 'должность': 'Ведущий аналитик',
 'максимальная зарплата': 500000,
 'минимальная зарплата': 250000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/67702365?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%